In [1]:
import io
import pandas as pd
from google.colab import files

#UPLOAD FILES

In [4]:
#Upload every file: dengue_features_train.csv, dengue_lables_train.csv, dengue_features_test.csv
archive = files.upload()

MessageError: ignored

In [5]:
df_features = pd.read_csv('dengue_features_train.csv', sep=',')
df_labels = pd.read_csv('dengue_labels_train.csv', sep=',')
train = pd.merge(df_features, df_labels, on=['city', 'year', 'weekofyear'])

train.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases
0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,297.742857,292.414286,299.8,295.9,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0,4
1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,293.951429,300.9,296.4,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,5
2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,298.878571,295.434286,300.5,297.3,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4,4
3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,299.228571,295.310000,301.4,297.0,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0,3
4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,299.664286,295.821429,301.9,297.5,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8,6


In [6]:
test = pd.read_csv('dengue_features_test.csv', sep=',')
test.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,2008,18,2008-04-29,-0.0189,-0.018900,0.102729,0.091200,78.60,298.492857,298.550000,294.527143,301.1,296.4,25.37,78.781429,78.60,15.918571,3.128571,26.528571,7.057143,33.3,21.7,75.2
1,sj,2008,19,2008-05-06,-0.0180,-0.012400,0.082043,0.072314,12.56,298.475714,298.557143,294.395714,300.8,296.7,21.83,78.230000,12.56,15.791429,2.571429,26.071429,5.557143,30.0,22.2,34.3
2,sj,2008,20,2008-05-13,-0.0015,NaN,0.151083,0.091529,3.66,299.455714,299.357143,295.308571,302.2,296.4,4.12,78.270000,3.66,16.674286,4.428571,27.928571,7.785714,32.8,22.8,3.0
3,sj,2008,21,2008-05-20,NaN,-0.019867,0.124329,0.125686,0.00,299.690000,299.728571,294.402857,303.0,296.9,2.20,73.015714,0.00,15.775714,4.342857,28.057143,6.271429,33.3,24.4,0.3
4,sj,2008,22,2008-05-27,0.0568,0.039833,0.062267,0.075914,0.76,299.780000,299.671429,294.760000,302.3,297.3,4.36,74.084286,0.76,16.137143,3.542857,27.614286,7.085714,33.3,23.3,84.1


#PREPROSESO

In [7]:
train.drop("week_start_date", axis=1, inplace=True)
test.drop("week_start_date", axis=1, inplace=True)

In [8]:
def remove_null_values(dataframe):
  if pd.isnull(dataframe).any().any():
    dataframe.fillna(method='ffill', inplace=True)

train.apply(remove_null_values)
test.apply(remove_null_values)

print(pd.isnull(train).any().any())
print(pd.isnull(test).any().any())


False
False


In [9]:
from sklearn import preprocessing

lb = preprocessing.LabelBinarizer()
train['city_bin'] = lb.fit_transform(train['city'])
test['city_bin'] = lb.fit_transform(test['city'])

In [55]:
# features = ['reanalysis_min_air_temp_k', 'reanalysis_relative_humidity_percent', 'station_avg_temp_c'] # 27.1082

features = ['reanalysis_avg_temp_k', 'reanalysis_relative_humidity_percent', 'station_avg_temp_c']

X_train = train[features]
X_test = test[features]

Y_train = train['total_cases']

1R 27.1082 ['reanalysis_min_air_temp_k', 'reanalysis_relative_humidity_percent', 'station_avg_temp_c']

#LINEAR REGRESSION


In [56]:
from sklearn import linear_model

model = linear_model.LinearRegression()
model.fit(X_train, Y_train)

model.coef_

array([ 0.51957806, -1.01512424,  4.86861083])

In [57]:
Y_prediction = model.predict(X_test)

In [58]:
import numpy as np

Y = np.rint(Y_prediction)
Y = Y.astype(int)
result = np.hstack(Y)

result

array([25, 23, 32, 39, 35, 35, 32, 38, 37, 35, 33, 38, 36, 40, 33, 40, 37,
       36, 39, 32, 32, 29, 33, 36, 29, 27, 35, 29, 31, 27, 28, 28, 26, 26,
       22, 22, 22, 23, 20, 20, 21, 20, 22, 23, 25, 23, 23, 22, 23, 24, 30,
       31, 28, 26, 21, 19, 25, 32, 31, 32, 38, 43, 40, 39, 39, 40, 41, 38,
       37, 39, 43, 34, 40, 38, 40, 43, 40, 37, 42, 38, 31, 31, 27, 30, 31,
       28, 30, 30, 24, 20, 10, 19, 23, 30, 26, 27, 28, 20, 22, 34, 34, 33,
       28, 23, 27, 34, 27, 29, 33, 28, 35, 36, 26, 25, 34, 36, 27, 23, 35,
       39, 33, 37, 38, 38, 33, 37, 32, 31, 31, 34, 38, 34, 27, 24, 31, 29,
       29, 23, 20, 23, 14, 14, 18, 19, 21, 22, 23, 16, 24, 21, 26, 22, 19,
       26, 26, 22, 28, 20, 26, 20, 14, 25, 19, 24, 30, 31, 26, 28, 28, 31,
       29, 29, 34, 29, 28, 35, 36, 25, 29, 35, 33, 33, 33, 30, 29, 28, 26,
       21, 25, 20, 27, 27, 25, 19, 24, 18, 19, 17, 22, 17, 26, 21, 22, 27,
       16, 20, 27, 24, 28, 29, 22, 26, 25, 32, 39, 43, 44, 45, 41, 33, 36,
       32, 38, 32, 34, 38

#RANDOM FOREST

In [38]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(criterion='mse', max_depth=100, random_state=0)

features = ['reanalysis_relative_humidity_percent', 'station_avg_temp_c', 'reanalysis_avg_temp_k']

regressor.fit(train[features], train['total_cases'])

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=100,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=0, splitter='best')

In [39]:
pd.DataFrame({'Attributes': features, 'Decision Tree': regressor.feature_importances_})

,Attributes,Decision Tree
0,reanalysis_relative_humidity_percent,0.431271
1,station_avg_temp_c,0.240856
2,reanalysis_avg_temp_k,0.327872


In [41]:
#regressor.fit(X=train.drop(['total_cases', 'city'], axis=1), y=train['total_cases'])

#y_pred = regressor.predict(X=train.drop(['total_cases', 'city'], axis=1))
#y_pred = regressor.fit(train[features], train['total_cases'])
#import numpy as np

#Y = np.rint(y_pred)
#Y = Y.astype(int)
#result = np.hstack(Y)

#result

#y_pred = dt.predict(X = f_test.drop(['count'], axis = 1))

#dt.fit( X = f_train.drop(['count'], axis=1), y = f_train['count'])

TypeError: ignored

#GENERATE OUTPUT

In [59]:
# generate output
output = pd.DataFrame({ 'city': test['city'], 'year': test['year'], 'weekofyear': test['weekofyear'], 'total_cases': result})


from google.colab import files

with open('result.csv', 'w') as f:
  output.to_csv(f,  index = False)
  

files.download('result.csv')

output.head()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,city,year,weekofyear,total_cases
0,sj,2008,18,25
1,sj,2008,19,23
2,sj,2008,20,32
3,sj,2008,21,39
4,sj,2008,22,35
